In [1]:
# %%
import numpy as np
#import scipy as spy
import scipy.integrate as integrate
import scipy.special as special
#from sympy import *
import matplotlib.pyplot as plt
# from py_functions import getalpha0

In [2]:
test = [[[1,1],[2,1],[3,1]],[[4,1],[5,1],[6,1]]]
print(np.shape(test))
cell_test = [None]*35
cell_test[2] = test


(2, 3, 2)


In [3]:

all_results = [None]*35

# %%
## Model Constants
Lp = 0.02 # [m] Initial Pouch Length - From original kellaris paper (2018)
perm0 = 8.85418782e-12#[m-3kg-1s4A2]
permr = 2.2 # for BOPP
Eb = 700e6 #[V/m] the breakdown strength of BOPP
w = 0.08 #[m] from original kellaris paper
t = 30e-6 #[m] Bopp thickness used in initial designs
k = 1000000 #[Nm-1] Achilles tendon average sourced from Litchwark 2005
rho_f = 903 #[kgm-3] Bopp Density TEKRA (2020)
rho_o = 920 #[kgm-3] FR3 Dielectric oil Density Cargill (2018)
alpf = np.pi/2 #[rad] Assume circular cross section on stroke completion
Eper = 0.6 # Percentage of electrode coverage
Va = 10000 # Actuation voltage
N = 1
nn = 7 # Number of series acutators
res = 1000

# %%
## Swept constants
#Va = np.linspace(0,10000,1000) # 0 -> 10kV res: 10V
E = Va/(2*t)
#Eper = np.linspace(0,1,0.01)

## model start
#Lambda for force
lamb = N*Va**2*perm0*permr*w/(4*t)

# Electrode length
Le = Lp*Eper

#optimal fill eqn based on electrode coverage ratio
# !!! MIGHT NOT WORK WITH ARRAY !!!
alpeqn = ((1-Eper)**2)*(2/np.pi) # Easy to calc constant
y = 0.01 # intial guess of angle
it = 0.0001 # iteration
scale = 1.6/it # how many possible
end = int(scale) # int for loop
for i in range(0,end): # over scale
    
    func = (y-np.sin(y)*np.cos(y))/(y**2) - alpeqn # Find value of remaining equation with embedded alpha0
    
    if func >= 0: # root is transition from negative to positive in this case
        alp0 = y - it # previous value before sign flip returned       
        break #root found, stop
    else:
        y += it #move to next guess
        continue


#Height
h = Lp*(np.sin(alp0)/alp0) # Height [m]

#Area w/ restraints 
# !!! MIGHT NOT WORK WITH ARRAY !!!
A = (Lp - Le)**2 /np.pi # Area [m2]
#Fill Volume 
Vol = A*w
#Blocking force
Fb = (lamb)*(np.cos(alp0)/(1-np.cos(alp0)))

Lpf = Lp - Le # Fully Deformed pouch length 

xmax = h - (Lpf*(np.sin(alpf)/alpf)) - Le # max displacement


alp = np.linspace(alp0,alpf,res) #sweep alpha and SET RESOLUTION

# Blocks of lp(alp) eqn sweep
lpnum = 2*A*np.square(alp)
lpdom = (alp - np.sin(alp)*np.cos(alp))
lp2 = lpnum/lpdom
lp = lp2**0.5
lp[0] = Lp # Removes unphysical discretisation error

# electrode length sweep
le = Lp - lp 

# displacement sweep
lpmp = np.sin(alp)/alp # lp multiplier in x
lplpmp = lp*lpmp
X = h - lplpmp - le

# Load of spring
P = k*X

# Force output of actuator
F = lamb*(np.cos(alp)/(1-np.cos(alp)))

# fig, ax = plt.subplots()
# ax.plot(X*1000, F)
# plt.xlabel(r"Tip Displacment $[mm]$")
# plt.ylabel(r"Maximum Load $[N]$")
# plt.title("Series Scaling test")
# %%
# For each actuator in series the load experienced for a given stroke is increased 
# by the mass of the actuators below it.
# All share the same blocking force 
# F - X Defines single actuator dynamics and are used to build series dynamics
out = [None]*4
f_normal = F/(w*t*perm0*permr*E*E) # Normalise force as per Kellaris 2019
strain = (X/(Lp*1))*100 # Calculate Strain
out[0] = X# Stack results for storage
out[1] = F
out[2] = strain 
out[3] = f_normal
all_results[0] = out
globalx = X # starts at full X as it will deform for sure as it did for single actuator
#addload = (nn-1)*9.81*(Vol*rho_o+Lp*w*t*2*rho_f) # additional load from actuators below current (nn)
#Look at load value 
xseries = np.zeros((nn,res))
mass = np.zeros((nn,res))
check1 = np.zeros((nn,res))
check2 = np.zeros((nn,res))


# CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
#                 '#f781bf', '#a65628', '#984ea3',
#                 '#999999', '#e41a1c', '#dede00']
# fig, ax = plt.subplots() 
for nn in range(2,8):
    for n in range(1,nn): #for every actuator in series
        # print('actuator')
        # print('n= ' + str(n))
        addload = N*(n-1)*9.81*(Vol*rho_o + Lp*w*t*2*rho_f)
        # print('addload|' + str(addload))
        for i in range(0,len(F)): # for each load value
        
            # print('load value')
            # print('i= ' + str(i))
            mass[n-1,i] = F[i] + addload  # establish an effective mass on single actuator FBD
            # print('mass|' + str(mass[n-1,i]))
            
            if mass[n-1,i] >= Fb: # If the actuator is blocked, move on to next load
                # print('Block|' +str(i) + ' ' + str(n))
                continue
            else: # the actuator can move but by how much?
                # the potential movement is given by graph in previous code
                # the place where the mass roughly equals force is the potential stroke of the actuator
                
                j=0
                while j  < len(F): # sweep F looking for cross point
                    
                    # We want to see the first point where mass > force as this gives us effective stroke
                    if F[j] <= mass[n-1,i]: # check above and if true insert the additional stroke  
                        # print('Effective|' +str(i) + ' ' + str(n))
                        # print('Sign change @' + str(j))
                        # print('Force here ' + str(F[j]))
                        # print('Effective mass ' +str(mass[n-1,i]))
                        
                        # print('x Effective ' + str(X[j]))
                        
                        xseries[n-1,i] = X[j]  # Additional displacement at this point is added to global
                        break
                    else:
                        j+=1
                continue
    XX = np.sum(xseries, axis = 0) #sum additional x for each actuator (row)
    #Corresponding displacement
    #build into array for every force
    #add to global

    
    # ax.plot(XX*1000,F, label = str(nn-1) + ' Actuator(s)', color = CB_color_cycle[nn-1])
    f_normal = F/(w*t*perm0*permr*E*E) # Normalise force as per Kellaris 2019
    strain = (XX/(Lp*nn))*100 # Calculate Strain
    out = [None]*4
    out[0] = X# Stack results for storage
    out[1] = F
    out[2] = strain 
    out[3] = f_normal
    all_results[nn-1] = out


In [4]:
# Parallel calculations - Literally just multiply force by N
for N in range(1,5):
    add_idx = 7*N # additional offset for list
    for i in range(0,7):
        use = all_results[i]
        use[1] = use[1]*N # Scale forces by No parallel in this part. 
        use[3] = use[3]*N # Same for normal
        all_results[i+add_idx] = use # plop it back in the right palce
        

In [5]:
specific_eng = [None]*35 # Empty list to  store specific energies
for i in range(0,35):
    F = all_results[i][1]
    X = all_results[i][0]    
    uuact = np.zeros(len(X))
    for j in range(0,res):
        uuact[j] = integrate.trapezoid(F[0:j],X[0:j]) # array of actuator energy till point
    specific_eng[i] = uuact

In [12]:
spring_ks = np.linspace(1000,1000000,100) # Lienar springs for block point finding. 
energys = [[None for j in range(100)] for i in range(35)]
# For each actuator
for i in range (0,35):
    use = all_results[i]
    x = use[0]
    f_act = use[1] 
    blocks = np.zeros((100,1),dtype=int)
    # For each spring, test and save block point 
    j = 0
    for k in spring_ks:
        for ii in range(0,len(x)):
            eng = np.zeros((3,1)) # format is [uact,usee,ratio]
            if k*x[ii] >= f_act[ii]:
                blocks[j] = ii
                eng[0] = integrate.trapezoid(f_act[0:ii],x[0:ii]) # Act Eng
                eng[1] = 0.5*k*x[ii]**2 # Spring Eng
                eng[2] = (eng[1]/eng[0])*100 # Energy storage ratio
                break 
            else:
                continue
        energys[i][j] = eng
        j += 1
        
# Run through x, find block point, what is the energy? 


array([[0.35824119],
       [0.02145337],
       [5.98852771]])